In [ ]:
import sys
print(sys.path)

In [1]:
import numpy as np
import pyeeg as pe
import pickle as pickle
import pandas as pd
import math
import mne
from sklearn import svm
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
import os
import time

In [2]:

channel = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18] #14 Channels chosen to fit Emotiv Epoch+
band = [4,8,12,16,25,45] #5 bands
window_size = 2 #Averaging band power of 2 sec
step_size = 0.1 #Each 0.1 sec update once
sample_rate = 500 #Sampling rate of 128 Hz
subjectList = ['01','02','03']
#List of subjects

In [3]:
import pandas as pd
import keras.backend as K
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.models import Sequential
from tensorflow.python.keras.layers.convolutional import Conv1D
from tensorflow.python.keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical 
from keras.layers import Flatten
from keras.layers import Dense
import numpy as np
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras import backend as K
from keras.models import Model
import timeit
from keras.models import Sequential
from tensorflow.python.keras.layers.core import Flatten, Dense, Dropout
from tensorflow.python.keras.layers.convolutional import Convolution1D, MaxPooling1D, ZeroPadding1D
from keras.optimizers import SGD
#import cv2, numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import mne
raw = mne.io.read_raw_eeglab("C:\Project_summary\EGGDataSet\EGGRealDataSet\sub-001\eeg\sub-001_task-eyesclosed_eeg.set",preload=False)
data,times=raw[:,:]
print(len(data))

In [ ]:
def FFT_Processing (sub, channel, band, window_size, step_size, sample_rate):
    '''
    arguments:  string subject#绝对路径,包含participants.tsv和derivative文件夹
                list channel indice
                list band
                int window size for FFT
                int step size for FFT
                int sample rate for FFT
    return:     void
    '''
    # meta = []
    #with open("/content/drive/My Drive/leading_ai/try/s" + sub + '.dat', 'rb') as file:
    # raw = mne.io.read_raw_eeglab("C:/Project_summary/EGGDataSet/EGGRealDataSet/derivatives/sub-001/eeg/sub-001_task-eyesclosed_eeg.set",preload=False)
    # if raw is not None :
    #     #subject = pickle.load(file, encoding='latin1') #resolve the python 2 data problem by encoding : latin1

    #     for i in range (0,40):
    #         # loop over 0-39 trails
    #         # data = subject["data"][i]#读取文件中的单个样本
    #         # labels = subject["labels"][i]#读取样本标签
    #         start = 0

    #         while start + window_size < raw[0][0].shape[1]:
    #             meta_array = []
    #             meta_data = [] #meta vector for analysis
    #             for j in channel:#读取不同通道
    #                 X = raw[j][0][start : start + window_size]
    #                 # X = data[j][start : start + window_size] #Slice raw data over 2 sec, at interval of 0.125 sec
    #                 Y = pe.bin_power(X, band, sample_rate) #FFT over 2 sec of channel j, in seq of theta, alpha, low beta, high beta, gamma
    #                 meta_data = meta_data + list(Y[0])

    #             meta_array.append(np.array(meta_data))
    #             meta_array.append(labels)

    #             meta.append(np.array(meta_array))    
    #             start = start + step_size
                
    #     meta = np.array(meta)
    #np.save('/content/drive/My Drive/leading_ai/try/s' + sub, meta, allow_pickle=True, fix_imports=True)
    meta = []
    df = pd.read_csv(sub+'participants.tsv', delimiter = '\t')

    # LabelDict={}
    # NumberOfData = len(df)
    # for k in range(NumberOfData):
    #     label = df.iloc[k,-2]
    #     if label not in LabelDict:
    #         LabelDict[label]=len(LabelDict)
    # print(LabelDict)

    for i in range(NumberOfData):
        print(i)
        raw = mne.io.read_raw_eeglab(sub+"derivatives/sub-{:03d}/eeg/sub-{:03d}_task-eyesclosed_eeg.set".format(i+1,i+1),preload=False)
        sfreq=raw.info['sfreq']
        data,_= raw[:,:]
        label = df.iloc[i,-2]
        start = 0
        while start + window_size*sfreq < len(data[0]):
            meta_array = []
            meta_data = [] #meta vector for analysis
            for j in channel:#读取不同通道 
                ChannelData = data[j]
                # print(ChannelData)
                # print(start, start + window_size*sfreq)
                X = ChannelData[int(start) : int(start + window_size*sfreq)] #Slice raw data over 2 sec, at interval of 0.125 sec
                Y = pe.bin_power(X, band, sample_rate) #FFT over 2 sec of channel j, in seq of theta, alpha, low beta, high beta, gamma
                meta_data = meta_data + list(Y[0])
            meta_array.append(np.array(meta_data))
            # labelCode = np.zeros(len(LabelDict))
            # labelCode[LabelDict[label]] = 1 
            # meta_array.append(labelCode)
            meta_array.append(label)
            meta.append(np.array(meta_array))    
            start = start + step_size * sfreq
    meta = np.array(meta)
    np.save('./nparray/meta.npy', meta, allow_pickle=True, fix_imports=True)

In [ ]:
FFT_Processing("C:/Project_summary/EGGDataSet/EGGRealDataSet/", channel=channel, band=band, window_size=window_size, step_size=step_size, sample_rate=sample_rate)

In [ ]:
data_training = []
label_training = []
data_testing = []
label_testing = []



with open('./nparray/meta.npy', 'rb') as file:
    sub = np.load(file,allow_pickle=True)
    for i in range (0,sub.shape[0]):
        if i % 5 == 0:
          data_testing.append(sub[i][0])
          label_testing.append(sub[i][1])
        else:
          data_training.append(sub[i][0])
          label_training.append(sub[i][1])


np.save('./DataAndLabels/data_training', np.array(data_training), allow_pickle=True, fix_imports=True)
np.save('./DataAndLabels/label_training', np.array(label_training), allow_pickle=True, fix_imports=True)
print("training dataset:", np.array(data_training).shape, np.array(label_training).shape)

np.save('./DataAndLabels/data_testing', np.array(data_testing), allow_pickle=True, fix_imports=True)
np.save('./DataAndLabels/label_testing', np.array(label_testing), allow_pickle=True, fix_imports=True)
print("testing dataset:", np.array(data_testing).shape, np.array(label_testing).shape)

In [4]:
with open('./DataAndLabels/data_training.npy', 'rb') as fileTrain:
    X  = np.load(fileTrain)
    
with open('./DataAndLabels/label_training.npy', 'rb') as fileTrainL:
    Y  = np.load(fileTrainL)
    
X = normalize(X)
Z = Y
print(X,Z)

[[0.14718135 0.06269111 0.05242517 ... 0.04258694 0.0520913  0.09663831]
 [0.12287107 0.07193411 0.05394435 ... 0.03949061 0.06694669 0.10783583]
 [0.11980121 0.06977577 0.05594874 ... 0.03924718 0.07156676 0.10738157]
 ...
 [0.10570603 0.07686313 0.04531252 ... 0.04259257 0.08191864 0.07792404]
 [0.12760023 0.08765102 0.04443266 ... 0.03925141 0.07168894 0.07462111]
 [0.13554783 0.07468287 0.02809172 ... 0.03004628 0.07509905 0.07695327]] [[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [5]:
# from keras.utils import to_categorical
# y_train = to_categorical(Z)
y_train = np.array(Z)
x_train = np.array(X[:])

In [6]:
with open('./DataAndLabels/data_testing.npy', 'rb') as fileTrain:
    M  = np.load(fileTrain)
    
with open('./DataAndLabels//label_testing.npy', 'rb') as fileTrainL:
    N  = np.load(fileTrainL)

M = normalize(M)
L = N
print(M,L)

[[0.11410238 0.08058227 0.06283299 ... 0.04662829 0.07533678 0.10683679]
 [0.1062635  0.08320407 0.06757253 ... 0.04018427 0.06642195 0.10408628]
 [0.15842044 0.08599591 0.07072215 ... 0.06662579 0.08338724 0.10151775]
 ...
 [0.13567688 0.0692273  0.05036737 ... 0.05641399 0.0773523  0.07916984]
 [0.12497286 0.060154   0.05001043 ... 0.05833618 0.06855889 0.08201546]
 [0.11371868 0.07288457 0.04826057 ... 0.04809681 0.07477087 0.07914009]] [[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [7]:
x_test = np.array(M[:])
y_test = np.array(L)

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

In [9]:
x_train = x_train.reshape(x_train.shape[0],x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0],x_test.shape[1], 1)

In [10]:
x_train.shape

(557260, 95, 1)

In [11]:
batch_size = 256
num_classes = 3
epochs = 200
input_shape=(1, x_train.shape[1], 1)

In [12]:
print(input_shape)

(1, 95, 1)


In [13]:
from keras.layers import Convolution1D, ZeroPadding1D, MaxPooling1D, BatchNormalization, Activation, Dropout, Flatten, Dense
from keras.regularizers import l2

In [14]:
model = Sequential()
input_shape=(None, x_train.shape[1], 1)
model.add(Conv1D(128, kernel_size=3,padding = 'same',activation='relu', input_shape=input_shape))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=(2)))
model.add(Conv1D(128,kernel_size=3,padding = 'same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=(2)))
model.add(Conv1D(64,kernel_size=3,padding = 'same', activation='relu'))
model.add(MaxPooling1D(pool_size=(2)))
model.add(Flatten())
model.add(Dense(64, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))
model.build(input_shape)
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 module_wrapper (ModuleWrapp  (None, 95, 128)          512       
 er)                                                             
                                                                 
 batch_normalization (BatchN  (None, 95, 128)          512       
 ormalization)                                                   
                                                                 
 max_pooling1d (MaxPooling1D  (None, 47, 128)          0         
 )                                                               
                                                                 
 module_wrapper_1 (ModuleWra  (None, 47, 128)          49280     
 pper)                                                           
                                                                 
 batch_normalization_1 (Batc  (None, 47, 128)          5

In [15]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history=model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,  
          verbose=1,validation_data=(x_test,y_test))